In [56]:
import pandas as pd

### Workbook to code 

#### mOVING ON TO THE BOX SCORES ####

# Example box score link
url_box = 'https://www.collegehockeynews.com/box/final/20230305/msu/ndm/'

# Example metrics link from same game
url_metrics = 'https://www.collegehockeynews.com/box/metrics.php?gd=96398'

# # output the entire html into a text file for review
# with open('../TEMP/box_score_example.txt', 'w') as file:
#     file.write(response.text)

# try to read with pandas
tables = pd.read_html(url_box)

tables[0] # score summary by period
tables[1] # shots by period
tables[2] # summary of penalties, power play results and faceoffs
tables[3] # scoring summary - period by period - periods are deliniated with a row with the "1st Period", "2nd Period", etc. text - if there is no row with this text, then there is no scoring in that period
tables[4] # penalty summary - period by period - periods are deliniated with a row with the "1st Period", "2nd Period", etc. text - if there is no row with this text, then there are no penalties in that period
tables[5] # goalie stats
tables[6] # visiting team stats
tables[7] # home team stats

,Michigan State,G,A,Pt.,+/-,Sh,PIM,FOW‑L
0,Matt Basgall,0,0,0,1,2,0,NaN
1,Tiernan Shoudy,1,0,1,1,4,0,11‑5
2,Daniel Russell,0,0,0,0,5,2,NaN
3,Viktor Hurtig,0,0,0,0,0,0,NaN
4,Karsen Dorwart,0,0,0,1,0,0,12‑10
5,Tanner Kelly,0,0,0,1,1,0,NaN
6,Erik Middendorf,0,1,1,2,0,0,NaN
7,Jesse Tucker,0,0,0,0,0,0,NaN
8,Jeremy Davidson,1,1,2,3,3,0,NaN
9,David Gucciardi,0,0,0,1,3,0,NaN


In [57]:
# tables.to_dict() # this is a dictionary of dataframes

tables[2].to_dict() # this is a dictionary of dataframes

# print(tables[2].to_string())

tables[7]


df_copy = tables[7].copy()

# Fill NaN values
df_copy.iloc[:, 7].fillna('0-0', inplace=True)
# replace the non standard dash with a standard dash
df_copy.replace('\u2011', '-', regex=True, inplace=True)
# Split the 'FOW‑L' column into two columns
df_copy[['FOW', 'FOL']] = df_copy.iloc[:, 7].str.split('-', expand=True)

df_copy

,Michigan State,G,A,Pt.,+/-,Sh,PIM,FOW‑L,FOW,FOL
0,Matt Basgall,0,0,0,1,2,0,0-0,0,0
1,Tiernan Shoudy,1,0,1,1,4,0,11-5,11,5
2,Daniel Russell,0,0,0,0,5,2,0-0,0,0
3,Viktor Hurtig,0,0,0,0,0,0,0-0,0,0
4,Karsen Dorwart,0,0,0,1,0,0,12-10,12,10
5,Tanner Kelly,0,0,0,1,1,0,0-0,0,0
6,Erik Middendorf,0,1,1,2,0,0,0-0,0,0
7,Jesse Tucker,0,0,0,0,0,0,0-0,0,0
8,Jeremy Davidson,1,1,2,3,3,0,0-0,0,0
9,David Gucciardi,0,0,0,1,3,0,0-0,0,0


In [58]:
tables[4]

df = tables[4].copy()
# save TEMP file
df.to_csv('../TEMP/box_score_example.csv')

OSError: Cannot save file into a non-existent directory: '..\TEMP'

In [ ]:


##################### NOT WORKING RIGHT NOW COME BACK TO LATER ##############################
def clean_penalty_summary(df):
    # Flatten multi-level columns if necessary
    df.columns = [' '.join(str(col)).strip() for col in df.columns.values]
    
    # Drop columns that are entirely NaN
    df.dropna(axis=1, how='all', inplace=True)
    
    # Initialize an empty list to store penalty events
    penalties = []
    
    # Loop through the DataFrame to create penalty event dictionaries
    period = None
    for i, row in df.iterrows():
        if 'Period' in row.values:
            period = row.values[0].split(' ')[0]
        else:
            penalty_event = {
                'Team': row[df.columns[0]],  # Adjust these to your actual column names
                'Period': period,
                'Player': row[df.columns[1]],
                'Length': row[df.columns[2]],
                'Type': row[df.columns[3]],
                'Time': row[df.columns[4]]
            }
            penalties.append(penalty_event)
    
    return penalties
###########################################

In [ ]:
df = tables[4].copy()

## Test code
df = clean_penalty_summary(df)

df

[{'Team': 'MSU',
  'Period': None,
  'Player': 'Daniel Russell',
  'Length': '2',
  'Type': 'Interference',
  'Time': '4:04'},
 {'Team': 'MSU',
  'Period': None,
  'Player': 'Jagger Joshua',
  'Length': '2',
  'Type': 'Interference',
  'Time': '16:42'},
 {'Team': '3rd Period',
  'Period': None,
  'Player': '3rd Period',
  'Length': '3rd Period',
  'Type': '3rd Period',
  'Time': '3rd Period'},
 {'Team': 'MSU',
  'Period': None,
  'Player': 'Nicolas Muller',
  'Length': '2',
  'Type': 'Hooking',
  'Time': '3:44'},
 {'Team': 'MSU',
  'Period': None,
  'Player': 'Michael Underwood',
  'Length': '2',
  'Type': 'Interference',
  'Time': '15:51'}]